<a href="https://colab.research.google.com/github/daichira-gif/daichira/blob/main/LangChain_GeminiProVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### APIキーの準備

In [ ]:
import pathlib
import textwrap
import google.generativeai as genai
from google.colab import userdata
from IPython.display import display
from IPython.display import Markdown

# Markdown出力
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# APIキーの準備
GOOGLE_API_KEY=userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
# パッケージのインストール

!pip install -q -U google-generativeai
!pip install Pillow
!pip install pdf2image

!apt-get install poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.3 [186 kB]
Fetched 186 kB in 1s (124 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121654 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.3_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.3) ...
Setting up poppler-utils (22.02.0-2ubuntu0.3) ...
Processing triggers for man-db (2.10.2-1) ...


### ファイルのアップロード

In [ ]:
from google.colab import files
import pandas as pd

# PDFファイルをアップロード
uploaded = files.upload()
# アップロードされたファイル名を取得
pdf_file = list(uploaded.keys())[0]

Saving 公共施設白書_市営住宅.pdf to 公共施設白書_市営住宅 (1).pdf


### PDFを画像に変換

In [ ]:
from pathlib import Path
from pdf2image import convert_from_path
from PIL import Image
#import pytesseract

# PDFファイルを画像に変換する関数
def pdf_to_images(pdf_path):
    images = convert_from_path(pdf_path)
    return images


# PDFファイルを指定
pdf_path = '/content/公共施設白書_市営住宅.pdf'

#output先のフォルダを作成
import os
folder = r"./image"

if not os.path.exists(folder):
  os.mkdir(folder)

# outputのファイルパス
img_path=Path("./image")

# convert_from_path関数を使ってPDFファイルを画像に変換
images = convert_from_path(pdf_path, output_folder=img_path, fmt='png')
images

[<PIL.PngImagePlugin.PngImageFile image mode=RGB size=2339x1653>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2339x1653>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2339x1653>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2339x1653>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2339x1653>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2339x1653>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2339x1653>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2339x1653>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2339x1653>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2339x1653>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2339x1653>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2339x1653>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2339x1653>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2339x1653>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2339x1653>,
 <PIL.PngI

# LangChainを使って、Google Geminiを動かしてみる
- 参考URL https://qiita.com/t_kamiya78/items/4fbb802a98a32bf4c5f7

### ライブラリのインストール

In [ ]:
# パッケージのインストール
!apt-get install tesseract-ocr-jpn
!apt-get install libtesseract-dev
!pip install Pillow==9.0.0
!pip install -U --quiet langchain-google-genai pillow
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-jpn tesseract-ocr-osd
0 upgraded, 4 newly installed, 0 to remove and 24 not upgraded.
Need to get 6,206 kB of archives.
After this operation, 18.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-jpn all 1:4.00~git30-7274cfa-1.1 [1,390 kB]
Fetched 6,206 kB in 3s (2,322 kB/s)
Selecting previously

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 6.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.2.0 which is incompatible.


### LangChain動作テスト

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model='gemini-pro', google_api_key = GOOGLE_API_KEY )

result = llm.invoke('日本一高い山はなんですか？')
print(result.content)
# 富士山

result = llm.invoke('Gemini PROの最大入力文字数と最大出力文字数を教えてください。')
print(result.content)
# Gemini PROは、最大入力文字数はなく、最大出力文字数は1024文字です。

result = llm.invoke('あなたについて教えてください。')
print(result.content)
# 私は大きな言語モデルであり、Googleによってトレーニングされました。私は膨大な量のテキストデータで訓練されており、人間のようなテキストを生成し、質問に答え、言語を翻訳することができます。私はまた、コードを記述したり、ストーリーを書いたり、詩を書いたりすることもできます。私はまだ開発中ですが、私はすでに多くの方法で人々を助けるために使用されています。例えば、私はカスタマーサービスのチャットボットとして、オンラインアシスタントとして、さらには医療診断ツールの開発に使用されています。
# 私は、機械学習と自然言語処理の分野における最近の進歩の産物です。私は、何十億もの単語で訓練されており、人間の言語を理解し、生成することができます。私はまた、複雑な質問に答えたり、テキストを翻訳したり、要約したりすることもできます。
# 私はまだ開発中ですが、私はすでに多くの方法で人々を助けるために使用されています。例えば、私はカスタマーサービスのチャットボットとして、オンラインアシスタントとして、さらには医療診断ツールの開発に使用されています。
# 私は、今後さらに多くの方法で人々を助けるために使用されることを楽しみにしています。私は、教育、医療、ビジネスなど、さまざまな分野で大きな影響を与えることができると信じています。

result = llm.invoke('いつまでの情報を知っていますか？')
print(result.content)
# 私の知識は、2021年4月までの情報までに限定されています。それ以降の情報を提供することはできません。最新の情報については、信頼できるニュースソースや政府機関のウェブサイトをご確認ください。



富士山
Gemini PROには入力文字数と出力文字数の制限はありません。
私は、Google によってトレーニングされた大規模な言語モデルです。私は、テキストやコード、翻訳、質問の回答など、さまざまなタスクを実行することができます。私は、膨大な量のテキストデータでトレーニングされており、その知識は日々更新されています。

私は、人々が情報を収集したり、タスクを完了したり、単に楽しんで過ごすのを手伝うためにここにいます。私は、いつも学習しており、常に新しいことを知ろうとしています。

私が得意なことをいくつかご紹介します。

* 文章の作成：私は、ブログ記事、ストーリー、その他の文章を作成することができます。私は、さまざまなスタイルやトーンで書くことができます。
* 翻訳：私は、100以上の言語を翻訳することができます。私は、正確かつ流暢な翻訳を行うことができます。
* コードの作成：私は、さまざまなプログラミング言語でコードを書くことができます。私は、単純なスクリプトから複雑なプログラムまで、あらゆるものを書くことができます。
* 質問の回答：私は、さまざまなトピックに関する質問に答えることができます。私は、正確かつ有益な回答を提供するよう努めています。
* 対話：私は、人々と自然な会話をすることができます。私は、質問に答えたり、ジョークを言ったり、単に雑談をすることができます。

私はまだ開発中であり、常に新しいことを学んでいます。私は、人々を助け、彼らの人生をより良くするために、最善を尽くしていきます。

何かご質問があれば、遠慮なくお尋ねください。
私の知識は2021年7月までしかありません。それ以降の最新情報は私が持っていませんので、ご自身で調べていただく必要があります。


In [ ]:
import base64

from langchain_core.messages import HumanMessage


## URL
#image_url = '<画像のURL>'
#message = HumanMessage(
#    content=[
#        {'type': 'image_url', 'image_url': image_url},
#    ]
#)

# base64エンコード

with open('/content/image/78857318-70bb-4b76-a839-5f2a247da31f-01.png', 'rb') as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode("utf-8")

message = HumanMessage(
    content=[
        {
            'type': 'image_url',
            'image_url': f'data:image/png;base64,${encoded_string}',
        },
    ]
)


In [ ]:
llm = ChatGoogleGenerativeAI(model='gemini-pro-vision', google_api_key = GOOGLE_API_KEY )

#imageフォルダから、任意のファイルパスを貼りつけ
with open('/content/image/78857318-70bb-4b76-a839-5f2a247da31f-01.png', 'rb') as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode("utf-8")

message = HumanMessage(
    content=[
        {
            'type': 'text',
            'text': 'この画像は、北九州市の市営住宅を表形式で示した一覧表です。この表をcsv形式で表示してください。',
        },
        {
            'type': 'image_url',
            'image_url': f'data:image/png;base64,${encoded_string}',
        },
    ]
)
res = llm.invoke([message])
print(res.content)

#質問１

message = HumanMessage(
    content=[
        {
            'type': 'text',
            'text': 'この表にある、「新松崎団地」の「区名」を教えてください。？',
        },
        {
            'type': 'image_url',
            'image_url': f'data:image/png;base64,${encoded_string}',
        },
    ]
)
res = llm.invoke([message])
print(res.content)

#質問２

message = HumanMessage(
    content=[
        {
            'type': 'text',
            'text': 'この表の中で、「湯川市民センター」の「町名」を教えてください。？',
        },
        {
            'type': 'image_url',
            'image_url': f'data:image/png;base64,${encoded_string}',
        },
    ]
)
res = llm.invoke([message])
print(res.content)

#質問３

message = HumanMessage(
    content=[
        {
            'type': 'text',
            'text': 'この表から、「北九州市エコタウンセンター」の「所管課」を教えてください。',
        },
        {
            'type': 'image_url',
            'image_url': f'data:image/png;base64,${encoded_string}',
        },
    ]
)
res = llm.invoke([message])
print(res.content)


 ```csv
"市営住宅名称","所在地","管理事務所","築年数","戸数","延床面積(㎡)","主要な構造","耐震改修","所有者","管理者","募集要件","入居率","備考"
"若松団地","北九州市若松区大字徳吉東１丁目１番１号","若松住宅管理事務所",1979,43,5409.58,"鉄骨鉄筋コンクリート造",未完了,北九州市,"北九州市住宅供給公社",住宅供給公社及び住宅管理課,64,65.6
"丸山団地","北九州市八幡西区丸山１丁目１番１号","八幡西住宅管理事務所",1968,59,12513.04,"鉄骨鉄筋コンクリート造",未完了,北九州市,"北九州市住宅供給公社",住宅供給公社及び住宅管理課,226,55.3
"大谷団地","北九州市八幡東区大谷１丁目１番１号","八幡東住宅管理事務所",1975,56,12475.36,"鉄骨鉄筋コンクリート造",未完了,北九州市,"北九州市住宅供給公社",住宅供給公社及び住宅管理課,158,74.7
"上本町団地（門司区）","北九州市門司区大字上本町１番１号","門司住宅管理事務所",1969,53,5371.65,"鉄骨鉄筋コンクリート造",未完了,北九州市,"北九州市住宅供給公社",住宅供給公社及び住宅管理課,115,68.2
"上本町団地（門司区）第２期","北九州市門司区大字上本町１番１号","門司住宅管理事務所",1975,47,5227.04,"鉄骨鉄筋コンクリート造",未完了,北九州市,"北九州市住宅供給公社",住宅供給公社及び住宅管理課,171,90.3
"庄司ヶ丘団地","北九州市小倉南区庄司ヶ丘１丁目１番１号","小倉南住宅管理事務所",1970,92,14090.53,"鉄骨鉄筋コンクリート造",未完了,北九州市,"北九州市住宅供給公社",住宅供給公社及び住宅管理課,687,51.5
"北方団地","北九州市門司区大字北方１丁目１番１号","門司住宅管理事務所",1968,46,4915.82,"鉄骨鉄筋コンクリート造",未完了,北九州市,"北九州市住宅供給公社",住宅供給公社及び住宅管理課,262,55
"日之出団地","北九州市門司区大字日之出１丁目１番１号","門司住宅管理事務所",2008,14,3450.05,"鉄骨鉄筋コンクリート造",完了,北九州市,"北